In [ ]:
"""
llama_multimodal_pipeline.py

Пример интеграции мультимодальной LLaMA (LLaVA / LLaMA-мультимодал) в существующий пайплайн `autocaption`.

Файл показывает:
- как обернуть мультимодальную модель в класс `LlamaMultimodal` с двумя методами:
    - `describe(image)` — генерирует несколько вариантов описания сцены
    - `answer_question(image, question)` — возвращает ответ на вопрос (VQA)
- как заменить существующие `PhotoDescriber` и `PhotoDescriberWithQuestion` на одну модель LLaMA

ВНИМАНИЕ:
- Конкретное имя модели (model_name) и способ загрузки зависят от того, какую реализацию мультимодальной LLaMA вы используете (LLaVA, MiniGPT-4, mLLaMA, etc.).
- В коде показаны 2 возможных бэкенда: "hf" (Hugging Face) и "api" (локальный/удалённый API типа Ollama/OpenAI). Выберите подходящий и укажите корректные параметры.

Зависимости (пример):
    pip install torch torchvision transformers pillow requests

Если у вас есть конкретная модель (ссылка на HF или endpoint Ollama), просто подставьте её в параметры при создании `LlamaMultimodal`.

"""

from __future__ import annotations
import base64
import io
import json
from typing import Optional, Dict, Any, List
from PIL import Image

# Опционально: зависимости для HF
try:
    from transformers import AutoProcessor, AutoModelForCausalLM
    from transformers import AutoTokenizer
    import torch
    HF_AVAILABLE = True
except Exception:
    HF_AVAILABLE = False

# Опционально: зависимости для API (requests)
import requests


class LlamaMultimodal:
    """Обёртка над мультимодальной LLaMA-реализацией.

    Поддерживает два режима работы:
      - mode='hf'  : загрузка модели через Hugging Face (локально / на сервере)
      - mode='api' : использование HTTP API (например, Ollama или ваш собственный endpoint)

    Настройте модель/endpoint под свою инфраструктуру.
    """

    def __init__(self, mode: str = "hf", model_name: Optional[str] = None, api_url: Optional[str] = None, device: str = "cpu"):
        self.mode = mode
        self.model_name = model_name
        self.api_url = api_url
        self.device = device

        if mode == "hf":
            if not HF_AVAILABLE:
                raise RuntimeError("transformers/torch недоступны — установите зависимости для HF режима")
            if model_name is None:
                raise ValueError("Укажите имя модели model_name для HF режима (модель должна поддерживать vision+LLM)")

            # Попытка загрузить процессор/модель. Пользователь должен указать корректную мультимодальную модель.
            # Пример (замените на вашу модель): model_name = "your-multimodal-llama-model"
            print(f"Загружаем HF модель: {model_name} ...")
            try:
                self.processor = AutoProcessor.from_pretrained(model_name)
                # Many multimodal HF models for LLaMA variants use AutoModelForCausalLM (или специализированный класс)
                self.model = AutoModelForCausalLM.from_pretrained(model_name)
                self.tokenizer = AutoTokenizer.from_pretrained(model_name)
                # Перекладываем на устройство
                if torch.cuda.is_available() and self.device.startswith("cuda"):
                    self.model.to(self.device)
                print("Модель HF загружена.")
            except Exception as e:
                raise RuntimeError(f"Не удалось загрузить HF модель: {e}")

        elif mode == "api":
            if api_url is None:
                raise ValueError("Укажите api_url для режима 'api' (например, http://localhost:11434 или ваш endpoint)")
            print(f"Использовать API endpoint: {api_url}")

        else:
            raise ValueError("mode должен быть 'hf' или 'api'")

    def _pil_to_base64(self, image: Image.Image) -> str:
        buffered = io.BytesIO()
        image.save(buffered, format="JPEG")
        return base64.b64encode(buffered.getvalue()).decode("utf-8")

    def describe(self, image: Image.Image, max_length: int = 256) -> Dict[str, str]:
        """Возвращает словарь с несколькими текстовыми описаниями.

        Формат результата:
            {
                'base': ...,
                'detailed': ...,
                'alternative': ...
            }
        """
        prompt_base = (
            "Опиши кратко, что изображено на фото (1-2 фразы).\n"
            "Далее — подробное описание сцены и элементов (3-4 предложения).\n"
            "Наконец — альтернативная формулировка краткого описания.\n"
            "Формат вывода: \nBASE: <...>\nDETAILED: <...>\nALTERNATIVE: <...>\n"
        )

        response_text = self._run_multimodal_prompt(image, prompt_base, max_length=max_length)

        # Парсим ответ по меткам. Если модель вернула произвольный текст — делаем best-effort разбор.
        parts = {"base": "", "detailed": "", "alternative": ""}
        try:
            # Ищем заранее ожидаемые маркеры
            for line in response_text.splitlines():
                if line.strip().upper().startswith("BASE:"):
                    parts['base'] = line.split(':', 1)[1].strip()
                elif line.strip().upper().startswith("DETAILED:"):
                    parts['detailed'] = line.split(':', 1)[1].strip()
                elif line.strip().upper().startswith("ALTERNATIVE:"):
                    parts['alternative'] = line.split(':', 1)[1].strip()

            # Если не разобрали — делим ответ на блоки по пустым строкам
            if not parts['base'] and not parts['detailed'] and not parts['alternative']:
                blocks = [b.strip() for b in response_text.split('\n\n') if b.strip()]
                if len(blocks) >= 1:
                    parts['base'] = blocks[0]
                if len(blocks) >= 2:
                    parts['detailed'] = blocks[1]
                if len(blocks) >= 3:
                    parts['alternative'] = blocks[2]

        except Exception:
            parts['base'] = response_text

        # Гарантии: если какое-то поле пустое, используем полный текст
        if not parts['base']:
            parts['base'] = response_text.split('\n')[0][:200]
        if not parts['detailed']:
            parts['detailed'] = response_text[:1000]
        if not parts['alternative']:
            parts['alternative'] = parts['base']

        return parts

    def answer_question(self, image: Image.Image, question: str, max_length: int = 128) -> str:
        """Отвечает на заданный вопрос про изображение.

        question — строка на любом языке (желательно на одном языке с системным промптом модели).
        """
        prompt = f"Вопрос: {question}\nКороткий и конкретный ответ:"
        response_text = self._run_multimodal_prompt(image, prompt, max_length=max_length)
        return response_text.strip()

    def _run_multimodal_prompt(self, image: Image.Image, prompt: str, max_length: int = 256) -> str:
        if self.mode == "hf":
            # Универсальный пример: процессор принимает images и текстовые подсказки.
            # Конкретные детали зависят от процессора/модели.
            try:
                inputs = self.processor(images=image, text=prompt, return_tensors="pt")
                # Перемещаем тензоры на устройство модели
                if hasattr(inputs, 'to'):
                    inputs = {k: v.to(self.device) for k, v in inputs.items()}

                # Генерация — зависит от реализации модели (некоторые мультимодальные модели используют generate)
                with torch.no_grad():
                    outputs = self.model.generate(**inputs, max_new_tokens=max_length)
                text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
                # Некоторые модели возвращают исходный промпт + ответ — уберём промпт, если он присутствует
                if prompt.strip() in text:
                    text = text.split(prompt, 1)[-1].strip()
                return text
            except Exception as e:
                raise RuntimeError(f"Ошибка при генерации HF: {e}")

        elif self.mode == "api":
            # Пример протокола для API: отправляем image как base64 + текстовый промпт.
            # Ожидается, что endpoint вернёт JSON {"text": "..."}
            b64 = self._pil_to_base64(image)
            payload = {
                "prompt": prompt,
                "image_base64": b64,
                "max_tokens": max_length
            }
            try:
                r = requests.post(self.api_url, json=payload, timeout=60)
                r.raise_for_status()
                data = r.json()
                # Предполагается поле 'text' или 'output'
                return data.get('text') or data.get('output') or json.dumps(data)
            except Exception as e:
                raise RuntimeError(f"Ошибка при обращении к API: {e}")
        else:
            raise RuntimeError("Неподдерживаемый режим")


# --- Интеграция с имеющимся pipeline ---
# Ниже пример адаптации вашей функции run_pipeline: заменяем describer и q_describer

from autocaption import ImageLoader, ImageRotator, CarDetector, ObjectExtractor, SceneExtractor


def run_pipeline_with_llama(image_path: List[str], source: bool, llama: LlamaMultimodal) -> List[List[Any]]:
    res = [[] for _ in range(len(image_path))]

    # инициализация классов (как у вас)
    print('Инициализация модели поворота')
    rotator = ImageRotator()
    print('Инициализация модели детектора автомобилей')
    car_detector = CarDetector()
    print('Инициализация модели нахождения объектов')
    object_extractor = ObjectExtractor()
    print('Инициализация модели классификации сцены')
    scene_classificator = SceneExtractor()

    print('Начинается обработка изображений (с LLaMA)')
    for i, path in enumerate(image_path):
        print(f"Обрабатывается изображение №{i+1}")
        image = ImageLoader(path=path, source=source).load_image()

        if image is None:
            res[i].append(f"Не удалось загрузить изображение: {path}.")
            continue

        image = rotator.rotate_image(image)
        if not car_detector.detect_car(image):
            res[i].append(f"Не удалось определить наличие автомобиля на изображении: {path}.")
            continue

        # 2. Нахождение признаков
        res[i].append(object_extractor.extract_features(image))

        temp_dict = scene_classificator.predict_scene(image)
        temp_dict.popitem()
        res[i].append(temp_dict)

        # --- заменяем генераторы описаний ---
        try:
            llama_desc = llama.describe(image)
            res[i].append(f"Базовое описание: {llama_desc.get('base')}")
            res[i].append(f"Подробное описание: {llama_desc.get('detailed')}")
            res[i].append(f"Альтернативное описание: {llama_desc.get('alternative')}")

            # VQA — пример вопроса. Вы можете менять вопрос в зависимости от задачи.
            vqa_q = "Где находится автомобиль? (коротко)"
            vqa_answer = llama.answer_question(image, vqa_q)
            res[i].append(f"Описание по вопросу (VQA): {vqa_answer}")

        except Exception as e:
            res[i].append(f"Ошибка при генерации описания LLaMA: {e}")

    return res


# --- Пример использования ---
if __name__ == "__main__":
    # Пример: HF режим (локальная загрузка) — замените model_name
    llama = LlamaMultimodal(mode='hf', model_name='your-multimodal-llama-model', device='cuda:0')

    # Пример: API режим (локальный Ollama / удалённый endpoint)
    llama = LlamaMultimodal(mode='api', api_url='http://localhost:8080/generate')

    # Пока просто создаём заглушку — пользователь должен выбрать конфигурацию
    print("Этот модуль демонстрирует, как подключить мультимодальную LLaMA к pipeline.\n"
          "Задайте режим и модель в блоке '__main__' перед запуском.")


# Конец файла
